# Brick optimisation first

Created 13/08/2024

Objectives:
* Implement brick pattern optimisation strategy to see if it actually works.

# Package imports

In [1]:
import sys
sys.path.append('../')

In [2]:
import h5py
from tenpy.tools import hdf5_io
import tenpy
import tenpy.linalg.np_conserved as npc

import os

In [3]:
from functools import reduce
from operator import mul

In [4]:
import numpy as np
import matplotlib.pyplot as plt

In [5]:
from SPTOptimization.SymmetryActionWithBoundaryUnitaries import SymmetryActionWithBoundaryUnitaries

from SPTOptimization.Optimizers.OneSiteSolver import OneSiteSolver

from SPTOptimization.utils import (
    to_npc_array,
    get_right_identity_environment
)

from SPTOptimization.Optimizers.utils import (
    one_site_optimization_sweep_right
)

In [6]:
import re

# Load data

In [7]:
DATA_DIR = r"../data/transverse_cluster_200_site_dmrg"

In [8]:
f_name = DATA_DIR + r"/0_90.h5"

In [9]:
f_name

'../data/transverse_cluster_200_site_dmrg/0_90.h5'

In [10]:
with h5py.File(f_name, 'r') as f:
    data = hdf5_io.load_from_hdf5(f)
    test_psi = data['wavefunction']

# Definitons

In [11]:
MAX_VIRTUAL_BOND_DIM = 8

In [12]:
np_I = np.array([[1,0],[0,1]])
np_X = np.array([[0,1],[1,0]])
np_Y = np.array([[0,-1j],[1j,0]])
np_Z = np.array([[1,0],[0,-1]])

In [13]:
npc_I = to_npc_array(np_I)
npc_X = to_npc_array(np_X)
npc_Y = to_npc_array(np_Y)
npc_Z = to_npc_array(np_Z)

In [14]:
test = SymmetryActionWithBoundaryUnitaries(
    test_psi,
    [np_X, np_I]*50
)

In [15]:
test.compute_svd_symmetry_action()

In [16]:
test.right_projected_symmetry_state

<npc.Array shape=(8, 8) labels=['vR', 'vR*']>

In [17]:
test.left_projected_symmetry_state

<npc.Array shape=(8, 8) labels=['vL', 'vL*']>

In [18]:
left_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=1
)

In [19]:
right_trivial_leg_charge = tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    [0,1],
    [[]],
    qconj=-1
)

## Functions

In [20]:
P_LEG_LABEL_REGEX_STRING = r"^p\d*$"
p_leg_pattern = re.compile(P_LEG_LABEL_REGEX_STRING)

def extract_p_leg_label_from_tensor(b):
    out = next(
        l for l in b.get_leg_labels()
        if (p_leg_pattern.match(l))
    )

    return out

def combine_b_tensors(b1, b2):
    b = npc.tensordot(b1, b2, ['vR', 'vL'])

    p1 = extract_p_leg_label_from_tensor(b1)
    p2 = extract_p_leg_label_from_tensor(b2)

    b = b.combine_legs([p1, p2])

    return b

def combine_b_tensors_lr(b1, b2):
    bl = b1.replace_label('p', 'pl')
    br = b2.replace_label('p', 'pr')

    b = npc.tensordot(bl, br, ['vR', 'vL'])
    b = b.combine_legs(['pl', 'pr'])

    return b

In [21]:
GROUPED_LEG_LABELS_REGEX_STRING = r"^\((\w+)\.(\w+)\)$"
grouped_leg_labels_pattern = re.compile(GROUPED_LEG_LABELS_REGEX_STRING)

def conjugate_leg_label(l):
    m = re.match(grouped_leg_labels_pattern, l)
    if m:
        left, right = m.groups()
        out = f"({left}*.{right}*)"
    else:
        out = l + '*'

    return out

In [22]:
def get_identity_operator(mps_tensor):
    p_leg_label = mps_tensor.get_leg_labels()[1]
    p_leg = mps_tensor.get_leg(p_leg_label)
    p_leg_label_conj = conjugate_leg_label(p_leg_label)

    out = npc.diag(
        1,
        leg=p_leg,
        dtype='complex',
        labels=[p_leg_label, p_leg_label_conj]
    )

    return out

In [23]:
def split_combined_b(b, max_virtual_bond_dim=MAX_VIRTUAL_BOND_DIM):
    t = b.split_legs()
    t = t.combine_legs([['vL', 'pl'], ['vR', 'pr']])

    U, S, VH = npc.svd(t, compute_uv=True, inner_labels=['vR', 'vL'])

    for i in range(U.shape[1]):
        U[:, i] *= S[i]
    
    U = U[:, :MAX_VIRTUAL_BOND_DIM]
    VH = VH[:MAX_VIRTUAL_BOND_DIM, :]

    bl = U.split_legs().replace_label('pl', 'p')
    br = VH.split_legs().replace_label('pr', 'p')

    bl.itranspose(['vL', 'p', 'vR'])
    br.itranspose(['vL', 'p', 'vR'])

    return bl, br

In [24]:
def split_combined_u(u):
    t = u.split_legs().combine_legs([['pl', 'pl*'], ['pr', 'pr*']])
    t.itranspose(['(pl.pl*)', '(pr.pr*)'])

    U, S, VH = npc.svd(t, compute_uv=True, inner_labels=['vR', 'vL'])

    for i in range(U.shape[1]):
        U[:, i] *= S[i]

    wl = (
        U
        .split_legs()
        .replace_labels(['pl', 'pl*'], ['p', 'p*'])
        .add_leg(left_trivial_leg_charge, 0, label='vL')
    )
    
    wr = (
        VH
        .split_legs()
        .replace_labels(['pr', 'pr*'], ['p', 'p*'])
        .add_leg(right_trivial_leg_charge, 0, label='vR')
    )

    return wl, wr

In [25]:
def unitary_to_mpo_tensor(u):
    out = (
        u
        .add_leg(left_trivial_leg_charge, 0, label='vL')
        .add_leg(right_trivial_leg_charge, 0, label='vR')
    )

    return out

In [111]:
PHYSICAL_LEG_LABELS = [
    'p',
    'p*',
    '(pl.pr)',
    '(pl*.pr*)'
]

In [113]:
def is_physical_leg_label(leg_label):
    return bool(leg_label in PHYSICAL_LEG_LABELS)

In [114]:
def get_physical_leg_labels(t):
    out = [
        ll for ll in t.get_leg_labels()
        if is_physical_leg_label(ll)
    ]

    return out

# Optimisation

## First right sweep

### Define tensors

In [26]:
test.right_symmetry_index

149

In [27]:
"""
bs = [
    test_psi.get_B(i).replace_label('p', f'p{k}')
    for k, i in enumerate(range(test.right_symmetry_index + 1, test.right_symmetry_index + 12))
]
"""

"\nbs = [\n    test_psi.get_B(i).replace_label('p', f'p{k}')\n    for k, i in enumerate(range(test.right_symmetry_index + 1, test.right_symmetry_index + 12))\n]\n"

In [28]:
bs = [
    test_psi.get_B(i)
    for k, i in enumerate(range(test.right_symmetry_index + 1, test.right_symmetry_index + 12))
]

In [29]:
len(bs)

11

In [30]:
grouped_bs = [(bs[0],),] + list(zip(bs[1::2], bs[2::2]))

In [31]:
grouped_bs

[(<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,),
 (<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>),
 (<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>),
 (<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>),
 (<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>),
 (<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>)]

In [32]:
combined_bs = [grouped_bs[0][0],] + [combine_b_tensors_lr(*p) for p in grouped_bs[1:]]

In [33]:
combined_bs

[<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 4, 8) labels=['vL', '(pl.pr)', 'vR']>,
 <npc.Array shape=(8, 4, 8) labels=['vL', '(pl.pr)', 'vR']>,
 <npc.Array shape=(8, 4, 8) labels=['vL', '(pl.pr)', 'vR']>,
 <npc.Array shape=(8, 4, 8) labels=['vL', '(pl.pr)', 'vR']>,
 <npc.Array shape=(8, 4, 8) labels=['vL', '(pl.pr)', 'vR']>]

Again, the naming convention could be an issue. Clash with single tensor and combined tensor.
Do we need to number the 'p' legs, or would using '(pl.pr)' be sufficient...?

In [34]:
unitaries = [
    get_identity_operator(t) for t in combined_bs
]

In [35]:
unitaries

[<npc.Array shape=(2, 2) labels=['p', 'p*']>,
 <npc.Array shape=(4, 4) labels=['(pl.pr)', '(pl*.pr*)']>,
 <npc.Array shape=(4, 4) labels=['(pl.pr)', '(pl*.pr*)']>,
 <npc.Array shape=(4, 4) labels=['(pl.pr)', '(pl*.pr*)']>,
 <npc.Array shape=(4, 4) labels=['(pl.pr)', '(pl*.pr*)']>,
 <npc.Array shape=(4, 4) labels=['(pl.pr)', '(pl*.pr*)']>]

In [36]:
combined_bs[1].conj()

<npc.Array shape=(8, 4, 8) labels=['vL*', '(pl*.pr*)', 'vR*']>

In [37]:
unitaries[0].to_ndarray()

array([[1.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j]])

In [38]:
unitaries[1].to_ndarray()

array([[1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j]])

In [39]:
expectations_r0 = list()

In [44]:
exps, _, tms = one_site_optimization_sweep_right(
    test.right_projected_symmetry_state,
    combined_bs,
    unitaries
)

expectations_r0.append(exps)

In [45]:
unitaries[0].to_ndarray()

array([[ 1.07021138e-06+0.j, -1.00000000e+00+0.j],
       [-1.00000000e+00+0.j, -1.07021138e-06+0.j]])

In [46]:
unitaries[1].to_ndarray()

array([[ 9.99077386e-01+0.j, -8.73910945e-04+0.j,  2.50287735e-02+0.j,
         3.48880106e-02+0.j],
       [-8.73910945e-04+0.j,  9.99077548e-01+0.j,  3.48868898e-02+0.j,
         2.50238804e-02+0.j],
       [ 2.50287735e-02+0.j,  3.48868898e-02+0.j, -9.99077425e-01+0.j,
         8.73910948e-04+0.j],
       [ 3.48880106e-02+0.j,  2.50238804e-02+0.j,  8.73910948e-04+0.j,
        -9.99077509e-01+0.j]])

In [47]:
expectations_r0

[[0.09774518810938414,
  1.177108211170415,
  1.1771082111704148,
  1.177108211170415,
  1.1771082111704148,
  1.1771082111704148],
 [1.1771082124804502,
  1.1771082125056047,
  1.1771082125056038,
  1.1771082125056036,
  1.177108212505603,
  1.177108212505603]]

In [35]:
unitaries

[<npc.Array shape=(2, 2) labels=['p*', 'p']>,
 <npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>,
 <npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>,
 <npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>,
 <npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>,
 <npc.Array shape=(4, 4) labels=['(pl.pr)', '(pl*.pr*)']>]

In [48]:
combined_bs_1 = list()

for b, u in zip(combined_bs, unitaries):
    ll = b.get_leg_labels()[1]
    llh = conjugate_leg_label(ll)

    new_b = npc.tensordot(b, u, [[ll,], [llh,]])

    combined_bs_1.append(new_b)

In [49]:
combined_bs_1

[<npc.Array shape=(8, 8, 2) labels=['vL', 'vR', 'p']>,
 <npc.Array shape=(8, 8, 4) labels=['vL', 'vR', '(pl.pr)']>,
 <npc.Array shape=(8, 8, 4) labels=['vL', 'vR', '(pl.pr)']>,
 <npc.Array shape=(8, 8, 4) labels=['vL', 'vR', '(pl.pr)']>,
 <npc.Array shape=(8, 8, 4) labels=['vL', 'vR', '(pl.pr)']>,
 <npc.Array shape=(8, 8, 4) labels=['vL', 'vR', '(pl.pr)']>]

In [50]:
combined_bs_1[1][:4, 5:]

<npc.Array shape=(4, 3, 4) labels=['vL', 'vR', '(pl.pr)']>

In [51]:
combined_bs_1[1][0,0,0]

(0.6058012307237871+0j)

In [52]:
b = combined_bs_1[0]
b.itranspose(['vL', 'p', 'vR'])

bs_1 = [b,]

In [53]:
for b in combined_bs_1[1:]:
    bl, br = split_combined_b(b)
    bs_1.append(bl)
    bs_1.append(br)

In [54]:
bs_1

[<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>]

Test overlap

In [89]:
t = test.right_projected_symmetry_state

for bu, bd in zip(bs_1, bs):
    t = npc.tensordot(t, bu, [['vR',],['vL',]])
    t = npc.tensordot(t, bd.conj(), [['vR*', 'p'],['vL*', 'p*']])

e = npc.trace(t)

In [90]:
e

(1.1770811213409618+0j)

Correct.

In [55]:
unitaries

[<npc.Array shape=(2, 2) labels=['p*', 'p']>,
 <npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>,
 <npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>,
 <npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>,
 <npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>,
 <npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>]

In [56]:
unitaries[1].split_legs()

<npc.Array shape=(2, 2, 2, 2) labels=['pl*', 'pr*', 'pl', 'pr']>

In [57]:
split_combined_u(unitaries[1])

(<npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>)

In [58]:
mpo_tensors_1 = [unitary_to_mpo_tensor(unitaries[0]),]

for u in unitaries[1:]:
    wl, wr = split_combined_u(u)
    mpo_tensors_1.append(wl)
    mpo_tensors_1.append(wr)    

In [59]:
mpo_tensors_1

[<npc.Array shape=(1, 1, 2, 2) labels=['vR', 'vL', 'p*', 'p']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>]

## Second layer

To-do:
* Get new grouped bs
* Initialise unitaries
    * Add default argument to optimisation for identity unitaries?
* Optimise unitaries
    * Repeatedly?
* Get new bs
* Get new mpo tensors
    * Compose and reduce function for mpo tensors

In [60]:
bs_1

[<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>]

In [66]:
grouped_bs_1

[(<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>),
 (<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>),
 (<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>),
 (<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>),
 (<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
  <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>),
 (<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,)]

In [96]:
combined_bs_1 = [combine_b_tensors_lr(*p) for p in grouped_bs_1[:-1]] + [grouped_bs_1[-1][0],]

In [97]:
combined_bs_1

[<npc.Array shape=(8, 4, 8) labels=['vL', '(pl.pr)', 'vR']>,
 <npc.Array shape=(8, 4, 8) labels=['vL', '(pl.pr)', 'vR']>,
 <npc.Array shape=(8, 4, 8) labels=['vL', '(pl.pr)', 'vR']>,
 <npc.Array shape=(8, 4, 8) labels=['vL', '(pl.pr)', 'vR']>,
 <npc.Array shape=(8, 4, 8) labels=['vL', '(pl.pr)', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>]

In [98]:
grouped_bs_bra_1 = list(zip(bs[:-1:2], bs[1:-1:2])) + [(bs[-1],),]

In [99]:
combined_bs_bra_1 = [combine_b_tensors_lr(*p) for p in grouped_bs_bra_1[:-1]] + [grouped_bs_bra_1[-1][0],]

In [100]:
combined_bs_bra_1

[<npc.Array shape=(8, 4, 8) labels=['vL', '(pl.pr)', 'vR']>,
 <npc.Array shape=(8, 4, 8) labels=['vL', '(pl.pr)', 'vR']>,
 <npc.Array shape=(8, 4, 8) labels=['vL', '(pl.pr)', 'vR']>,
 <npc.Array shape=(8, 4, 8) labels=['vL', '(pl.pr)', 'vR']>,
 <npc.Array shape=(8, 4, 8) labels=['vL', '(pl.pr)', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>]

Test overlap

In [101]:
t = test.right_projected_symmetry_state

for bu, bd in zip(combined_bs_1[:-1], combined_bs_bra_1[:-1]):
    t = npc.tensordot(t, bu, [['vR',],['vL',]])
    t = npc.tensordot(t, bd.conj(), [['vR*', '(pl.pr)'],['vL*', '(pl*.pr*)']])


t = npc.tensordot(t, combined_bs_1[-1], [['vR',],['vL',]])
t = npc.tensordot(t, combined_bs_bra_1[-1].conj(), [['vR*', 'p'],['vL*', 'p*']])

e = npc.trace(t)

In [102]:
e

(1.1770811213409613+0j)

In [103]:
unitaries = [
    get_identity_operator(t) for t in combined_bs_1
]

In [104]:
expectations_r1 = list()

In [107]:
exps, *_ = one_site_optimization_sweep_right(
    test.right_projected_symmetry_state,
    combined_bs_1,
    unitaries,
    combined_bs_bra_1
)

expectations_r1.append(exps)

In [108]:
expectations_r1

[[1.177215623040653,
  1.1766780855010426,
  1.1766765270856747,
  1.1766764781457462,
  1.1766764779116077,
  1.1766764779078547],
 [1.1772144067036576,
  1.1766780432988435,
  1.1766765337487242,
  1.1766764849438134,
  1.1766764847081512,
  1.176676484703498]]

In [109]:
unitaries

[<npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>,
 <npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>,
 <npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>,
 <npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>,
 <npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>,
 <npc.Array shape=(2, 2) labels=['p*', 'p']>]

In [110]:
unitaries[0].to_ndarray()

array([[ 9.99800514e-01+0.j,  9.23001401e-03+0.j,  1.65253595e-04+0.j,
        -1.77119007e-02+0.j],
       [-9.23004351e-03+0.j,  9.99800514e-01+0.j,  1.77119240e-02+0.j,
         1.63588146e-04+0.j],
       [ 1.61813358e-04+0.j, -1.77119404e-02+0.j,  9.99800494e-01+0.j,
         9.23222468e-03+0.j],
       [ 1.77119171e-02+0.j,  1.63478807e-04+0.j, -9.23219518e-03+0.j,
         9.99800494e-01+0.j]])

In [115]:
combined_bs_2 = list()

for b, u in zip(combined_bs_1, unitaries):
    ll = get_physical_leg_labels(b)[0]
    llh = conjugate_leg_label(ll)

    new_b = npc.tensordot(b, u, [[ll,], [llh,]])

    combined_bs_2.append(new_b)

In [116]:
combined_bs_2

[<npc.Array shape=(8, 8, 4) labels=['vL', 'vR', '(pl.pr)']>,
 <npc.Array shape=(8, 8, 4) labels=['vL', 'vR', '(pl.pr)']>,
 <npc.Array shape=(8, 8, 4) labels=['vL', 'vR', '(pl.pr)']>,
 <npc.Array shape=(8, 8, 4) labels=['vL', 'vR', '(pl.pr)']>,
 <npc.Array shape=(8, 8, 4) labels=['vL', 'vR', '(pl.pr)']>,
 <npc.Array shape=(8, 8, 2) labels=['vL', 'vR', 'p']>]

In [118]:
bs_2 = list()

In [119]:
for b in combined_bs_2[:-1]:
    bl, br = split_combined_b(b)
    bs_2.append(bl)
    bs_2.append(br)

bs_2.append(combined_bs_2[-1])

In [120]:
bs_2

[<npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 2, 8) labels=['vL', 'p', 'vR']>,
 <npc.Array shape=(8, 8, 2) labels=['vL', 'vR', 'p']>]

Test overlap

In [121]:
t = test.right_projected_symmetry_state

for bu, bd in zip(bs_2, bs):
    t = npc.tensordot(t, bu, [['vR',],['vL',]])
    t = npc.tensordot(t, bd.conj(), [['vR*', 'p'],['vL*', 'p*']])

e = npc.trace(t)

In [122]:
e

(1.1766159438778185+0j)

Correct.

In [55]:
unitaries

[<npc.Array shape=(2, 2) labels=['p*', 'p']>,
 <npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>,
 <npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>,
 <npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>,
 <npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>,
 <npc.Array shape=(4, 4) labels=['(pl*.pr*)', '(pl.pr)']>]

In [56]:
unitaries[1].split_legs()

<npc.Array shape=(2, 2, 2, 2) labels=['pl*', 'pr*', 'pl', 'pr']>

In [57]:
split_combined_u(unitaries[1])

(<npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>)

In [58]:
mpo_tensors_1 = [unitary_to_mpo_tensor(unitaries[0]),]

for u in unitaries[1:]:
    wl, wr = split_combined_u(u)
    mpo_tensors_1.append(wl)
    mpo_tensors_1.append(wr)    

In [59]:
mpo_tensors_1

[<npc.Array shape=(1, 1, 2, 2) labels=['vR', 'vL', 'p*', 'p']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>,
 <npc.Array shape=(1, 2, 2, 4) labels=['vL', 'p', 'p*', 'vR']>,
 <npc.Array shape=(1, 4, 2, 2) labels=['vR', 'vL', 'p', 'p*']>]

# Old code

In [61]:
right_leg_charge

LegCharge(ChargeInfo([], []), qconj=-1,
array([0, 4]), array([], shape=(1, 0), dtype=int64))

In [62]:
right_leg_charge.chinfo

ChargeInfo([], [])

In [65]:
right_leg_charge.slices

array([0, 4])

In [64]:
type(right_leg_charge.slices)

numpy.ndarray

In [66]:
right_leg_charge.slices.shape

(2,)

In [67]:
right_leg_charge.charges

array([], shape=(1, 0), dtype=int64)

In [75]:
tenpy.linalg.charges.LegCharge(
    tenpy.linalg.charges.ChargeInfo([], []),
    np.array([0, 4]),
    np.array([[]], dtype='int64'),
    qconj=-1
)

LegCharge(ChargeInfo([], []), qconj=-1,
array([0, 4]), array([], shape=(1, 0), dtype=int64))

In [74]:
np.array([[]], dtype='int64')

array([], shape=(1, 0), dtype=int64)